# Data Acquisition Example: Testing WLAN Client Receiver System Noise

Test data for WLAN is implemented with the the iperf support in [ssmdevices](https://gitlab.nist.gov/ssm/ssmdevices). It produces many more columns of data than we need, so we just use the two defined in `ipc_columns`: throughput and a timestamp.

The results of the tests and the corresponding test conditions are stored in a [flat database](https://en.wikipedia.org/wiki/Flat_file_database) stored in SQLite format. It is implemented as a typical intended use case of `labbench.RelationalDataInSQLite`. The test conditions are implemented as states in the attenuators, `iperf`, and `wlan`, so all we need to log these results into the database is to add the `db.on_set` call. Any states that are changed after that function call (for example, with the for loops in the acquisition code) become columns in the database; these values are automatically kept up to date and written to the database on calls to `db.write`.

In [1]:
# This should be run in python 3.6.x from the computer connected to the AP side

%pylab inline
import time,os
import ssmdevices as ssm
import labbench as lb
import pandas as pd

def meshpoints (x1, x2, *args):
    ''' For input of N-dimensions of grid point vectors (x1, x2, ..., xN),
        where each (x1, x2, ... xN) has dimension (M1, M2, ..., MN),
        returns an array with shape (M1*M2*...*MN,N) that consists of the
        unique points in the multi-dimensional grid with sampled at
        the vector points (x1, x2, ..., xN) in each axis. This is also known
        as the the cartesian product of (x1 ... xN).
    '''
    ret = np.array(np.meshgrid(*((x1,x2)+args)))
    return ret.reshape([ret.shape[0],np.prod(ret.shape[1:])]).T

setup_time = 5 # seconds
acquisition_time = 10 #  seconds

iperf_columns       = ['iperf_bits_per_second','iperf_timestamp']
sweep_points       = meshpoints(list(range(55,76,2)),[110]+list(range(0,21,2)))
#np.random.shuffle(c_e0_points) # randomize?

# db_path = '{}'.format(time.strftime("%Y-%m-%d-%HH-%MM"))

lb.show_messages('info')

c = ssm.instruments.MiniCircuitsRCDAT('11604210008')
e0 = ssm.instruments.MiniCircuitsRCDAT('11604210014')
iperf_client = ssm.software.IPerf('10.0.0.2', bind='10.0.0.3', port=5010, interval=0.1)
iperf_server = ssm.software.IPerf(bind='10.0.0.2', port=5010)
wlan = ssm.software.WLANStatus('Wi-Fi', ssid='EnGenius1')
db = lb.StatesToSQLite(time.strftime("%Y-%m-%d-%H%M%S no shuffle"))

states = {}

def update(update):
    states[update['name']] = update['new']

import time
t0 = time.time()

# This `with' block ensures sockets and iperf subprocesses are
# closed correctly when the script ends, even if exceptions are raised
with lb.concurrently(c, e0, iperf_client, iperf_server, wlan, db):
    try:
        # Log all state changes in lte_laa and each client in iperf_clients
        db.observe_states([c,e0,iperf_server,iperf_client,wlan])

        # Only need to do this if there is a variable attenuator in the test setup
        # lte_enb_atten.state.attenuation = 20. # dB

        wlan.interface_disconnect()
        c.state.attenuation = 30
        e0.state.attenuation = 110

        iperf_server.start()
        iperf_client.start()

        lb.logger.info('Pausing for {}s setup time'.format(setup_time))
        lb.sleep(setup_time)
        
        df=iperf_client.fetch()
        df_server=iperf_server.fetch()

        iperf_server.kill()
        iperf_client.kill()

        row = {}
        i = 0
        
        for c.state.attenuation, e0.state.attenuation in sweep_points:
            p = iperf_server.settings.port
            iperf_client.settings.port = iperf_server.settings.port = p+1            
            
            i += 1
            lb.logger.info('{}/{}: atten C {} dB, atten E0 {} dB'\
                           .format(i, len(sweep_points), c.state.attenuation, e0.state.attenuation))

            try:
                wlan.interface_disconnect(5)
            except TimeoutError as e:
                lb.logger.debug(str(e))
                lb.logger.debug('Failed to disconnect from AP, continuing anyway')
                
            # Do the sequence
            try:
                wlan.interface_connect(5)
            except TimeoutError as e:
                lb.logger.info(str(e))
                lb.logger.info('No connection to AP - skipping iperf')
                df = None                
            else:
                iperf_server.start()
                iperf_client.start()

                lb.sleep(acquisition_time)

                df = iperf_client.fetch()
                df_server = iperf_server.fetch()
                
                lb.logger.info('iperf_client returned {} rows'.format(len(df)))
                lb.logger.info('iperf_server returned {} rows'.format(len(df_server)))
                
                iperf_server.kill()
                iperf_client.kill()
                
                # This refresh state for signal and the rest of WLAN
                wlan.state.signal
                
            finally:
                db.append(iperf_client=df)
                db.write()
    finally:
        iperf_server.kill()
        iperf_client.kill()

# # # Load the sqlite database and save a copy into a csv
df = lb.read(os.path.join(db.path, 'master.db'))
df.to_csv(os.path.join(db.path,'master.csv'))

Populating the interactive namespace from numpy and matplotlib


2018-10-29 15:30:59.138 - INFO - Connected in 0.92s
2018-10-29 15:31:00.414 - INFO - Pausing for 5s setup time
C:\Users\dkuester\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
2018-10-29 15:31:05.494 - WARNING - IPerf('10.0.0.2') - tried kill(), but no process is running
2018-10-29 15:31:05.516 - INFO - 1/132: atten C 55.0 dB, atten E0 110.0 dB
2018-10-29 15:31:16.936 - INFO - iperf_client returned 96 rows
2018-10-29 15:31:16.936 - INFO - iperf_server returned 38 rows
2018-10-29 15:31:17.074 - INFO - 2/132: atten C 57.0 dB, atten E0 110.0 dB
2018-10-29 15:31:29.056 - INFO - iperf_client returned 98 rows
2018-10-29 15:31:29.057 - INFO - iperf_server returned 38 rows
2018-10-29 15:31:29.146 - INFO - 3/132: atten C 59.0 dB, atten E0 110.0 dB
2018-10-29 15:31:41.185 - INFO - iperf_client returned 98 rows

In [2]:
df

,c_attenuation,e0_attenuation,host_log,host_time,iperf_client,wlan_bssid_1,wlan_channel,wlan_description,wlan_encryption,wlan_radio_status,wlan_radio_type,wlan_signal,wlan_state
id,,,,,,,,,,,,,
0,55.0,110.0,0 2018-10-29 153116.987264\host_log.txt,2018-10-29 15:31:16.987264,0 2018-10-29 153116.987264\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,60,connected
1,57.0,110.0,1 2018-10-29 153129.96379\host_log.txt,2018-10-29 15:31:29.96379,1 2018-10-29 153129.96379\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,58,connected
2,59.0,110.0,2 2018-10-29 153141.241353\host_log.txt,2018-10-29 15:31:41.241353,2 2018-10-29 153141.241353\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,55,connected
3,61.0,110.0,3 2018-10-29 153153.339029\host_log.txt,2018-10-29 15:31:53.339029,3 2018-10-29 153153.339029\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,48,connected
4,63.0,110.0,4 2018-10-29 153205.473118\host_log.txt,2018-10-29 15:32:05.473118,4 2018-10-29 153205.473118\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,48,connected
5,65.0,110.0,5 2018-10-29 153217.597366\host_log.txt,2018-10-29 15:32:17.597366,5 2018-10-29 153217.597366\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,45,connected
6,67.0,110.0,6 2018-10-29 153229.705271\host_log.txt,2018-10-29 15:32:29.705271,6 2018-10-29 153229.705271\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,44,connected
7,69.0,110.0,7 2018-10-29 153241.826709\host_log.txt,2018-10-29 15:32:41.826709,7 2018-10-29 153241.826709\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,42,connected
8,71.0,110.0,8 2018-10-29 153254.141515\host_log.txt,2018-10-29 15:32:54.141515,8 2018-10-29 153254.141515\iperf_client.csv,88:dc:96:37:67:c6,36,Realtek 8812AU Wireless LAN 802.11ac USB NIC,None,None,802.11a,42,connected
